In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import geopandas as gpd
import pandas as pd
import branca
from siuba import *

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
import bb_rt_helper

In [ ]:
import importlib
importlib.reload(bb_rt_helper)

# 100 Recs - D7

## 1 - Corridor Improvements

In [ ]:
bb_rt_helper.fm_from_bbutils?

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='07 - Los Angeles', category='corridor')

In [ ]:
corr_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    corr_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
m_corr = fm_dict[182].segment_speed_map()

In [ ]:
m_corr

## Quick SHOPP

## Route 4 (SHN only)

In [ ]:
shape_id = '40319_FEB22'
stop_range = [33, 43]

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].set_filter(route_names=['4'])

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
corr_4 = fm_dict[182].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors = [corr_4]

## Draft Corridor Recs, LA Metro, D7

* Route 207, Slauson - Hollywood
* Route 4, Sepulveda - Hoover
* Route 233, Ventura - Woodman

## Route 207

In [ ]:
shape_id = '2070290_FEB22'
stop_range = [20, 57]

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].set_filter(route_names=['207'])

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
corr_207 = fm_dict[182].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors = [corr_207]

## Route 4 (Full)

In [ ]:
# shape_id = '40319_FEB22'
# stop_range = [3, 50]
# fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)
# fm_dict[182].set_filter(route_names=['4'])

In [ ]:
# fm_dict[182].corridor_metrics()

In [ ]:
# fm_dict[182].quick_map_corridor()

In [ ]:
# corr_4 = fm_dict[182].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor
# corridors += [corr_4]

## Route 233

In [ ]:
shape_id = '2330157_FEB22'
stop_range = [17, 48]

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].set_filter(route_names=['233'])

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
corr_233 = fm_dict[182].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors += [corr_233]

## Route 108

In [ ]:
shape_id = '1080303_FEB22'
stop_range = [4, 52]

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].set_filter(route_names=['108'])

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
corr_108 = fm_dict[182].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors += [corr_108]

## Route 260

In [ ]:
shape_id = '2600271_FEB22'
stop_range = [4, 39]

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].set_filter(route_names=['260'])

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
corr_260 = fm_dict[182].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors += [corr_260]

## 2 - Hotspot Improvements

In [ ]:
# without get_sorted returns mainly Metro Rail
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='07 - Los Angeles', category='hotspot', get_sorted = True, fm_dict = fm_dict)

In [ ]:
hs_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    hs_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
m_hs = fm_dict[182].segment_speed_map()

In [ ]:
m_hs

# Draft Hotspot Recs, LA Metro, D7

## Route 4 (I-405)

In [ ]:
shape_id = '40320_FEB22'
stop_range = [77, 81]

In [ ]:
fm_dict[182].set_filter(route_names=['4'])

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
hs_4 = fm_dict[182].corridor.copy()
hotspots = [hs_4]

## Route 217 (La Cienega)

In [ ]:
shape_id = '2170310_FEB22'
stop_range = [18, 22]

In [ ]:
fm_dict[182].set_filter(route_names=['217'])

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
hs_217 = fm_dict[182].corridor.copy()
hotspots += [hs_217]

## Route 4 (Beverly Hills)

In [ ]:
shape_id = '40314_FEB22'
stop_range = [20, 21]

In [ ]:
fm_dict[182].set_filter(route_names=['4'])

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
hs_4a = fm_dict[182].corridor.copy()
hotspots += [hs_4a]

## Route 720/20 (Beverly Hills)

In [ ]:
shape_id = '200782_FEB22'
stop_range = [13, 21]

In [ ]:
fm_dict[182].set_filter(route_names=['720', '20'])

In [ ]:
fm_dict[182].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[182].corridor_metrics()

In [ ]:
fm_dict[182].quick_map_corridor()

In [ ]:
hs_20 = fm_dict[182].corridor.copy()
hotspots += [hs_20]

# Draft Hotspot Recs, Long Beach Transit, D7

In [ ]:
fm_dict[170].segment_speed_map()

## Route 121 (Livingston - PCH)

In [ ]:
shape_id = '1210150'
stop_range = [26, 33]

In [ ]:
fm_dict[170].set_filter(route_names=['121'])

In [ ]:
fm_dict[170].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[170].corridor_metrics()

In [ ]:
fm_dict[170].quick_map_corridor()

In [ ]:
hs_121 = fm_dict[170].corridor.copy()
hotspots += [hs_121]

## Route 51 (Anaheim - PCH)

In [ ]:
shape_id = '510074'
stop_range = [6, 10]

In [ ]:
fm_dict[170].set_filter(route_names=['51'])

In [ ]:
fm_dict[170].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[170].corridor_metrics()

In [ ]:
fm_dict[170].quick_map_corridor()

In [ ]:
hs_51 = fm_dict[170].corridor.copy()
hotspots += [hs_51]

# Draft Hotspot Recs, LADOT, D7

In [ ]:
fm_dict[183].segment_speed_map(how = 'average')

## Route Boyle Heights (Mott - Cornwell)

In [ ]:
shape_id = '15715'
stop_range = [20, 22]

In [ ]:
fm_dict[183].set_filter(route_names=['Boyle Heights'])

In [ ]:
fm_dict[183].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[183].corridor_metrics()

In [ ]:
fm_dict[183].quick_map_corridor()

In [ ]:
hs_boyle = fm_dict[183].corridor.copy()
hotspots += [hs_boyle]

## Route Hollywood (Mott - Cornwell)

In [ ]:
shape_id = '9255'
stop_range = [44, 45]

In [ ]:
fm_dict[183].reset_filter()

In [ ]:
fm_dict[183].set_filter(route_names=['Hollywood Counterclockwise', 'Hollywood Clockwise'])

In [ ]:
fm_dict[183].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[183].corridor_trip_speeds >> distinct(_.route_short_name)

In [ ]:
fm_dict[183].corridor_metrics()

In [ ]:
fm_dict[183].quick_map_corridor()

In [ ]:
hs_holly = fm_dict[183].corridor.copy()
hotspots += [hs_holly]

## Santa Clarita Transit

* Identified routes are single-trip, school services, out of scope

In [ ]:
# fm_dict[295].segment_speed_map()

## Map Testing

In [ ]:
all_speed_segs = pd.concat(hs_speed_views)
all_corr_segs = pd.concat(corr_speed_views)
# full join to preserve all unique segments, since they can differ
# from hotspot to corridor (or indeed across operators)
all_segs = all_speed_segs >> full_join(_, all_corr_segs)

In [ ]:
all_corr = pd.concat(corridors)
all_corr['location_type'] = 'corridor'
all_hs = pd.concat(hotspots)
all_hs['location_type'] = 'hotspot'
combined = pd.concat([all_corr, all_hs])

In [ ]:
spatial = all_segs.to_crs('EPSG:3310').sjoin(combined, how='inner', predicate = 'intersects')

In [ ]:
ids = spatial >> distinct(_.route_id)

In [ ]:
all_segs = all_segs >> filter(_.route_id.isin(ids.route_id))

In [ ]:
bb_rt_helper.bb_map_all(hotspots, corridors, all_segs, district = '07 - Los Angeles')